# MNIST

## Читаем данные

In [2]:
import numpy as np, tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
sns.set_palette("colorblind")
palette = sns.color_palette()
figsize = (10, 10)
legend_fontsize = 16

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Логистическая регрессия

In [4]:
x = tf.placeholder(tf.float32, [None, 784]) 
y_ = tf.placeholder(tf.float32, [None, 10])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

y = tf.nn.softmax(tf.matmul(x, W) + b)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [6]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [7]:
for i in range(1001):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    if i % 50 == 0:
        print("[%d]\ttrain accuracy=%.6f\ttest accuracy=%.6f" % (i,
            sess.run(accuracy, feed_dict={x: mnist.train.images, y_: mnist.train.labels}),
            sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})
        ))

[0]	train accuracy=0.409855	test accuracy=0.407500
[50]	train accuracy=0.864945	test accuracy=0.872500
[100]	train accuracy=0.886691	test accuracy=0.894800
[150]	train accuracy=0.885164	test accuracy=0.892600
[200]	train accuracy=0.897000	test accuracy=0.903100
[250]	train accuracy=0.891636	test accuracy=0.898900
[300]	train accuracy=0.903200	test accuracy=0.907400
[350]	train accuracy=0.902982	test accuracy=0.906300
[400]	train accuracy=0.900709	test accuracy=0.903700
[450]	train accuracy=0.904891	test accuracy=0.910200
[500]	train accuracy=0.910000	test accuracy=0.912500
[550]	train accuracy=0.901055	test accuracy=0.902200
[600]	train accuracy=0.912182	test accuracy=0.916500
[650]	train accuracy=0.914655	test accuracy=0.916500
[700]	train accuracy=0.915673	test accuracy=0.915900
[750]	train accuracy=0.914927	test accuracy=0.917300
[800]	train accuracy=0.914200	test accuracy=0.916100
[850]	train accuracy=0.913473	test accuracy=0.914000
[900]	train accuracy=0.912764	test accuracy=0.917

## Двухуровневая сеть с ReLU

In [14]:
x = tf.placeholder(tf.float32, [None, 784]) 
y_ = tf.placeholder(tf.float32, [None, 10])

W_relu = tf.Variable(tf.truncated_normal([784, 2000], stddev=0.1))
b_relu = tf.Variable(tf.truncated_normal([2000], stddev=0.1))

W = tf.Variable(tf.zeros([2000, 10]))
b = tf.Variable(tf.zeros([10]))

h = tf.nn.relu(tf.matmul(x, W_relu) + b_relu)

logit = tf.matmul(h, W) + b
y = tf.nn.softmax(logit)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logit, labels=y_))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [16]:
for i in range(1001):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    if i % 50 == 0:
        print("[%d]\ttest accuracy=%.6f\ttrain accuracy=%.6f" % (i,
            sess.run(accuracy, feed_dict={x: mnist.train.images, y_: mnist.train.labels}),
            sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})
        ))

[0]	test accuracy=0.098255	train accuracy=0.098000
[50]	test accuracy=0.899145	train accuracy=0.904500
[100]	test accuracy=0.928764	train accuracy=0.931400
[150]	test accuracy=0.940782	train accuracy=0.940800
[200]	test accuracy=0.948909	train accuracy=0.947600
[250]	test accuracy=0.953527	train accuracy=0.949900
[300]	test accuracy=0.958436	train accuracy=0.956200
[350]	test accuracy=0.958018	train accuracy=0.951700
[400]	test accuracy=0.963782	train accuracy=0.959600
[450]	test accuracy=0.964855	train accuracy=0.960600
[500]	test accuracy=0.968564	train accuracy=0.963500
[550]	test accuracy=0.970418	train accuracy=0.965300
[600]	test accuracy=0.971691	train accuracy=0.962800
[650]	test accuracy=0.973891	train accuracy=0.968000
[700]	test accuracy=0.976473	train accuracy=0.968500
[750]	test accuracy=0.976473	train accuracy=0.968800
[800]	test accuracy=0.977327	train accuracy=0.968500
[850]	test accuracy=0.979964	train accuracy=0.971200
[900]	test accuracy=0.979764	train accuracy=0.969

## Добавим дропаут

In [17]:
x = tf.placeholder(tf.float32, [None, 784]) 
y_ = tf.placeholder(tf.float32, [None, 10])

W_relu = tf.Variable(tf.truncated_normal([784, 200], stddev=0.1))
b_relu = tf.Variable(tf.truncated_normal([200], stddev=0.1))

W = tf.Variable(tf.zeros([200, 10]))
b = tf.Variable(tf.zeros([10]))

h = tf.nn.relu(tf.matmul(x, W_relu) + b_relu)

keep_probability = tf.placeholder(tf.float32)
h_drop = tf.nn.dropout(h, keep_probability)

logit = tf.matmul(h_drop, W) + b
y = tf.nn.softmax(logit)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logit, labels=y_))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [18]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [19]:
for i in range(1001):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, keep_probability : 0.5})
    if i % 50 == 0:
        print("[%d]\ttest accuracy=%.6f\ttrain accuracy=%.6f" % (i,
            sess.run(accuracy, feed_dict={x: mnist.train.images, y_: mnist.train.labels, keep_probability : 1.0}),
            sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_probability : 1.0})
        ))

[0]	test accuracy=0.111255	train accuracy=0.108900
[50]	test accuracy=0.819909	train accuracy=0.824400
[100]	test accuracy=0.908727	train accuracy=0.912500
[150]	test accuracy=0.913709	train accuracy=0.916900
[200]	test accuracy=0.928055	train accuracy=0.929800
[250]	test accuracy=0.933491	train accuracy=0.934500
[300]	test accuracy=0.937000	train accuracy=0.937800
[350]	test accuracy=0.941891	train accuracy=0.943200
[400]	test accuracy=0.941582	train accuracy=0.939800
[450]	test accuracy=0.946200	train accuracy=0.946300
[500]	test accuracy=0.950564	train accuracy=0.949200
[550]	test accuracy=0.952982	train accuracy=0.954100
[600]	test accuracy=0.953818	train accuracy=0.953600
[650]	test accuracy=0.956382	train accuracy=0.955200
[700]	test accuracy=0.957782	train accuracy=0.955200
[750]	test accuracy=0.960982	train accuracy=0.959600
[800]	test accuracy=0.959327	train accuracy=0.957100
[850]	test accuracy=0.962382	train accuracy=0.960900
[900]	test accuracy=0.962236	train accuracy=0.959

# MNIST на Keras

## Предобработка

In [20]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.utils import np_utils
from keras import callbacks
from keras import optimizers, regularizers
tf.python.control_flow_ops = tf

Using TensorFlow backend.


In [21]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
X_train = x_train.reshape(x_train.shape[0], x_train.shape[1] * x_train.shape[2])
X_test = x_test.reshape(x_test.shape[0], x_test.shape[1] * x_test.shape[2])
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

batch_size = 100

In [22]:
X_train.shape

(60000, 784)

## Логистическая регрессия

In [23]:
model = Sequential()
model.add(Dense(10, input_shape=(784,)))
model.add(Activation('softmax'))

In [24]:
sgd = optimizers.SGD(lr=0.5)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [25]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=20,
          verbose=1, validation_data=(X_test, Y_test),
         callbacks=[callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1),
#                    ModelCheckpoint(kfold_weights_path, monitor='val_loss', save_best_only=True, verbose=0)
                   ])

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 3s - loss: 0.3966 - acc: 0.8854 - val_loss: 0.3031 - val_acc: 0.9145
Epoch 2/20
60000/60000 [==============================] - 2s - loss: 0.3060 - acc: 0.9138 - val_loss: 0.2857 - val_acc: 0.9208
Epoch 3/20
60000/60000 [==============================] - 2s - loss: 0.2919 - acc: 0.9179 - val_loss: 0.2790 - val_acc: 0.9200
Epoch 4/20
60000/60000 [==============================] - 2s - loss: 0.2842 - acc: 0.9202 - val_loss: 0.2828 - val_acc: 0.9206
Epoch 5/20
60000/60000 [==============================] - 2s - loss: 0.2794 - acc: 0.9214 - val_loss: 0.2769 - val_acc: 0.9233
Epoch 6/20
60000/60000 [==============================] - 2s - loss: 0.2757 - acc: 0.9234 - val_loss: 0.2821 - val_acc: 0.9211
Epoch 7/20
60000/60000 [==============================] - 2s - loss: 0.2724 - acc: 0.9242 - val_loss: 0.2725 - val_acc: 0.9237
Epoch 8/20
60000/60000 [==============================] - 2s 

In [27]:
model = Sequential()
model.add(Dense(10, input_shape=(784,)))
model.add(Activation('softmax'))
sgd = optimizers.SGD(lr=0.5)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.fit(
    X_train, Y_train,
    batch_size=batch_size,
    epochs=10,
    verbose=1,
    validation_data=(X_test, Y_test),
    callbacks=[callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 2s - loss: 0.3913 - acc: 0.8877 - val_loss: 0.3251 - val_acc: 0.9072
Epoch 2/10
60000/60000 [==============================] - 2s - loss: 0.3065 - acc: 0.9131 - val_loss: 0.3010 - val_acc: 0.9131
Epoch 3/10
60000/60000 [==============================] - 2s - loss: 0.2917 - acc: 0.9182 - val_loss: 0.2797 - val_acc: 0.9204
Epoch 4/10
60000/60000 [==============================] - 2s - loss: 0.2844 - acc: 0.9205 - val_loss: 0.2715 - val_acc: 0.9241
Epoch 5/10
60000/60000 [==============================] - 2s - loss: 0.2782 - acc: 0.9229 - val_loss: 0.2835 - val_acc: 0.9216
Epoch 6/10
60000/60000 [==============================] - 2s - loss: 0.2752 - acc: 0.9230 - val_loss: 0.2753 - val_acc: 0.9227
Epoch 7/10
60000/60000 [==============================] - 2s - loss: 0.2723 - acc: 0.9241 - val_loss: 0.2705 - val_acc: 0.9226
Epoch 8/10
60000/60000 [==============================] - 2s 

## Сеть с одним скрытым слоем

In [28]:
model = Sequential()
model.add(Dense(200, input_shape=(784,)))
model.add(Activation('sigmoid'))
model.add(Dense(10))
model.add(Activation('softmax'))

In [29]:
sgd = optimizers.SGD(lr=0.1)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [30]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=10,
          verbose=1, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 5s - loss: 0.8433 - acc: 0.8019 - val_loss: 0.4354 - val_acc: 0.8863
Epoch 2/10
60000/60000 [==============================] - 4s - loss: 0.4008 - acc: 0.8901 - val_loss: 0.3474 - val_acc: 0.9053
Epoch 3/10
60000/60000 [==============================] - 5s - loss: 0.3438 - acc: 0.9024 - val_loss: 0.3171 - val_acc: 0.9076
Epoch 4/10
60000/60000 [==============================] - 5s - loss: 0.3173 - acc: 0.9088 - val_loss: 0.2953 - val_acc: 0.9164
Epoch 5/10
60000/60000 [==============================] - 5s - loss: 0.2997 - acc: 0.9134 - val_loss: 0.2852 - val_acc: 0.9194
Epoch 6/10
60000/60000 [==============================] - 4s - loss: 0.2863 - acc: 0.9174 - val_loss: 0.2714 - val_acc: 0.9218
Epoch 7/10
60000/60000 [==============================] - 4s - loss: 0.2746 - acc: 0.9210 - val_loss: 0.2638 - val_acc: 0.9250
Epoch 8/10
60000/60000 [==============================] - 4s 

In [33]:
model = Sequential()
model.add(Dense(200, input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))

In [34]:
sgd = optimizers.SGD(lr=0.1)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [35]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=10,
          verbose=1, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 5s - loss: 0.4160 - acc: 0.8880 - val_loss: 0.2528 - val_acc: 0.9300
Epoch 2/10
60000/60000 [==============================] - 4s - loss: 0.2333 - acc: 0.9344 - val_loss: 0.2012 - val_acc: 0.9399
Epoch 3/10
60000/60000 [==============================] - 5s - loss: 0.1830 - acc: 0.9487 - val_loss: 0.1624 - val_acc: 0.9531
Epoch 4/10
60000/60000 [==============================] - 4s - loss: 0.1511 - acc: 0.9574 - val_loss: 0.1413 - val_acc: 0.9590
Epoch 5/10
60000/60000 [==============================] - 4s - loss: 0.1288 - acc: 0.9642 - val_loss: 0.1221 - val_acc: 0.9642
Epoch 6/10
60000/60000 [==============================] - 5s - loss: 0.1125 - acc: 0.9688 - val_loss: 0.1111 - val_acc: 0.9660
Epoch 7/10
60000/60000 [==============================] - 5s - loss: 0.0994 - acc: 0.9724 - val_loss: 0.1037 - val_acc: 0.9698
Epoch 8/10
60000/60000 [==============================] - 5s 

## Что если добавить слоёв?

In [36]:
model = Sequential()
model.add(Dense(200, input_shape=(784,), kernel_regularizer=regularizers.l2(0.001)))
model.add(Activation('relu'))
model.add(Dense(200, input_shape=(784,), kernel_regularizer=regularizers.l2(0.001)))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))

In [37]:
sgd = optimizers.SGD(lr=0.1)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [38]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=10,
          verbose=1, validation_data=(X_test, Y_test),
         callbacks=[callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 10s - loss: 0.8952 - acc: 0.8825 - val_loss: 0.6618 - val_acc: 0.9350
Epoch 2/10
60000/60000 [==============================] - 9s - loss: 0.6011 - acc: 0.9423 - val_loss: 0.5256 - val_acc: 0.9530
Epoch 3/10
60000/60000 [==============================] - 9s - loss: 0.4827 - acc: 0.9567 - val_loss: 0.4428 - val_acc: 0.9587
Epoch 4/10
60000/60000 [==============================] - 9s - loss: 0.4008 - acc: 0.9657 - val_loss: 0.3730 - val_acc: 0.9664
Epoch 5/10
60000/60000 [==============================] - 9s - loss: 0.3405 - acc: 0.9704 - val_loss: 0.3219 - val_acc: 0.9689
Epoch 6/10
60000/60000 [==============================] - 9s - loss: 0.2936 - acc: 0.9737 - val_loss: 0.2843 - val_acc: 0.9704
Epoch 7/10
60000/60000 [==============================] - 7s - loss: 0.2569 - acc: 0.9769 - val_loss: 0.2550 - val_acc: 0.9728
Epoch 8/10
60000/60000 [==============================] - 9s

## Сеть с дропаутом

In [34]:
model = Sequential()
model.add(Dense(200, input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

In [35]:
sgd = optimizers.SGD(lr=0.1)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [39]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=10,
          verbose=1, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 9s - loss: 0.1716 - acc: 0.9836 - val_loss: 0.1828 - val_acc: 0.9766
Epoch 2/10
60000/60000 [==============================] - 9s - loss: 0.1604 - acc: 0.9838 - val_loss: 0.1721 - val_acc: 0.9770
Epoch 3/10
60000/60000 [==============================] - 9s - loss: 0.1503 - acc: 0.9849 - val_loss: 0.1660 - val_acc: 0.9769
Epoch 4/10
60000/60000 [==============================] - 9s - loss: 0.1421 - acc: 0.9849 - val_loss: 0.1585 - val_acc: 0.9779
Epoch 5/10
60000/60000 [==============================] - 9s - loss: 0.1360 - acc: 0.9854 - val_loss: 0.1548 - val_acc: 0.9764
Epoch 6/10
60000/60000 [==============================] - 11s - loss: 0.1303 - acc: 0.9865 - val_loss: 0.1522 - val_acc: 0.9771
Epoch 7/10
60000/60000 [==============================] - 10s - loss: 0.1258 - acc: 0.9865 - val_loss: 0.1435 - val_acc: 0.9787
Epoch 8/10
60000/60000 [==============================] - 1

## Правильная инициализация

In [45]:
model = Sequential()
model.add(Dense(200, kernel_initializer='he_uniform', input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))

In [46]:
sgd = optimizers.SGD(lr=0.1)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [47]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=10,
          verbose=1, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 5s - loss: 0.4127 - acc: 0.8879 - val_loss: 0.2642 - val_acc: 0.9250
Epoch 2/10
60000/60000 [==============================] - 5s - loss: 0.2318 - acc: 0.9345 - val_loss: 0.1935 - val_acc: 0.9421
Epoch 3/10
60000/60000 [==============================] - 5s - loss: 0.1821 - acc: 0.9486 - val_loss: 0.1594 - val_acc: 0.9527
Epoch 4/10
60000/60000 [==============================] - 5s - loss: 0.1498 - acc: 0.9579 - val_loss: 0.1390 - val_acc: 0.9590
Epoch 5/10
60000/60000 [==============================] - 5s - loss: 0.1282 - acc: 0.9636 - val_loss: 0.1241 - val_acc: 0.9635
Epoch 6/10
60000/60000 [==============================] - 4s - loss: 0.1120 - acc: 0.9688 - val_loss: 0.1182 - val_acc: 0.9648
Epoch 7/10
60000/60000 [==============================] - 5s - loss: 0.0993 - acc: 0.9722 - val_loss: 0.1037 - val_acc: 0.9682
Epoch 8/10
60000/60000 [==============================] - 5s 

In [48]:
model = Sequential()
model.add(Dense(200, kernel_initializer='glorot_uniform', input_shape=(784,)))
model.add(Activation('sigmoid'))
model.add(Dense(10))
model.add(Activation('softmax'))

In [49]:
sgd = optimizers.SGD(lr=0.1)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [50]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=10,
          verbose=1, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 5s - loss: 0.8350 - acc: 0.8046 - val_loss: 0.4321 - val_acc: 0.8878
Epoch 2/10
60000/60000 [==============================] - 5s - loss: 0.3984 - acc: 0.8904 - val_loss: 0.3423 - val_acc: 0.9044
Epoch 3/10
60000/60000 [==============================] - 5s - loss: 0.3424 - acc: 0.9023 - val_loss: 0.3111 - val_acc: 0.9116
Epoch 4/10
60000/60000 [==============================] - 5s - loss: 0.3160 - acc: 0.9096 - val_loss: 0.2943 - val_acc: 0.9146
Epoch 5/10
60000/60000 [==============================] - 5s - loss: 0.2987 - acc: 0.9138 - val_loss: 0.2805 - val_acc: 0.9208
Epoch 6/10
60000/60000 [==============================] - 5s - loss: 0.2853 - acc: 0.9181 - val_loss: 0.2722 - val_acc: 0.9215
Epoch 7/10
60000/60000 [==============================] - 5s - loss: 0.2734 - acc: 0.9213 - val_loss: 0.2614 - val_acc: 0.9259
Epoch 8/10
60000/60000 [==============================] - 5s 

## Сеть с нормализацией по мини-батчам

In [51]:
model = Sequential()
model.add(Dense(200, input_shape=(784,)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dense(10))
model.add(Activation('softmax'))

In [52]:
sgd = optimizers.SGD(lr=0.1)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [53]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=10,
          verbose=1, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s - loss: 0.2249 - acc: 0.9340 - val_loss: 0.1295 - val_acc: 0.9610
Epoch 2/10
60000/60000 [==============================] - 7s - loss: 0.1042 - acc: 0.9703 - val_loss: 0.0991 - val_acc: 0.9692
Epoch 3/10
60000/60000 [==============================] - 7s - loss: 0.0746 - acc: 0.9783 - val_loss: 0.0778 - val_acc: 0.9748
Epoch 4/10
60000/60000 [==============================] - 7s - loss: 0.0555 - acc: 0.9848 - val_loss: 0.0816 - val_acc: 0.9726
Epoch 5/10
60000/60000 [==============================] - 6s - loss: 0.0450 - acc: 0.9872 - val_loss: 0.0772 - val_acc: 0.9748
Epoch 6/10
60000/60000 [==============================] - 7s - loss: 0.0362 - acc: 0.9894 - val_loss: 0.0748 - val_acc: 0.9772
Epoch 7/10
60000/60000 [==============================] - 6s - loss: 0.0289 - acc: 0.9924 - val_loss: 0.0745 - val_acc: 0.9755
Epoch 8/10
60000/60000 [==============================] - 7s 

## Сравним оптимизаторы на сети с одним скрытым слоем ReLU

In [54]:
model = Sequential()
model.add(Dense(200, input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))

In [55]:
sgd = optimizers.SGD(lr=0.1)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [56]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=10,
          verbose=1, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 5s - loss: 0.4166 - acc: 0.8880 - val_loss: 0.2502 - val_acc: 0.9292
Epoch 2/10
60000/60000 [==============================] - 5s - loss: 0.2332 - acc: 0.9343 - val_loss: 0.1974 - val_acc: 0.9416
Epoch 3/10
60000/60000 [==============================] - 5s - loss: 0.1857 - acc: 0.9478 - val_loss: 0.1649 - val_acc: 0.9512
Epoch 4/10
60000/60000 [==============================] - 5s - loss: 0.1542 - acc: 0.9558 - val_loss: 0.1401 - val_acc: 0.9583
Epoch 5/10
60000/60000 [==============================] - 5s - loss: 0.1325 - acc: 0.9618 - val_loss: 0.1260 - val_acc: 0.9632
Epoch 6/10
60000/60000 [==============================] - 5s - loss: 0.1155 - acc: 0.9672 - val_loss: 0.1155 - val_acc: 0.9656
Epoch 7/10
60000/60000 [==============================] - 5s - loss: 0.1025 - acc: 0.9710 - val_loss: 0.1039 - val_acc: 0.9700
Epoch 8/10
60000/60000 [==============================] - 5s 

In [57]:
model = Sequential()
model.add(Dense(200, input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))
sgd = optimizers.SGD(lr=0.1, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [58]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=10,
          verbose=1, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 6s - loss: 0.2376 - acc: 0.9290 - val_loss: 0.1136 - val_acc: 0.9638
Epoch 2/10
60000/60000 [==============================] - 5s - loss: 0.0958 - acc: 0.9704 - val_loss: 0.0909 - val_acc: 0.9723
Epoch 3/10
60000/60000 [==============================] - 5s - loss: 0.0671 - acc: 0.9789 - val_loss: 0.0832 - val_acc: 0.9748
Epoch 4/10
60000/60000 [==============================] - 5s - loss: 0.0496 - acc: 0.9841 - val_loss: 0.0810 - val_acc: 0.9746
Epoch 5/10
60000/60000 [==============================] - 5s - loss: 0.0359 - acc: 0.9883 - val_loss: 0.0735 - val_acc: 0.9776
Epoch 6/10
60000/60000 [==============================] - 5s - loss: 0.0275 - acc: 0.9916 - val_loss: 0.0802 - val_acc: 0.9761
Epoch 7/10
60000/60000 [==============================] - 5s - loss: 0.0217 - acc: 0.9932 - val_loss: 0.0755 - val_acc: 0.9784
Epoch 8/10
60000/60000 [==============================] - 5s 

In [59]:
model = Sequential()
model.add(Dense(200, input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))
sgd = optimizers.SGD(lr=0.1, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [60]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=10,
          verbose=1, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 5s - loss: 0.2211 - acc: 0.9339 - val_loss: 0.1294 - val_acc: 0.9605
Epoch 2/10
60000/60000 [==============================] - 5s - loss: 0.0928 - acc: 0.9722 - val_loss: 0.0850 - val_acc: 0.9738
Epoch 3/10
60000/60000 [==============================] - 6s - loss: 0.0628 - acc: 0.9809 - val_loss: 0.0821 - val_acc: 0.9747
Epoch 4/10
60000/60000 [==============================] - 6s - loss: 0.0492 - acc: 0.9843 - val_loss: 0.0783 - val_acc: 0.9771
Epoch 5/10
60000/60000 [==============================] - 5s - loss: 0.0374 - acc: 0.9885 - val_loss: 0.0778 - val_acc: 0.9763
Epoch 6/10
60000/60000 [==============================] - 5s - loss: 0.0271 - acc: 0.9916 - val_loss: 0.0744 - val_acc: 0.9790
Epoch 7/10
60000/60000 [==============================] - 5s - loss: 0.0219 - acc: 0.9929 - val_loss: 0.0819 - val_acc: 0.9783
Epoch 8/10
60000/60000 [==============================] - 5s 

In [61]:
model = Sequential()
model.add(Dense(200, input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))
sgd = optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [62]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=10,
          verbose=1, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s - loss: 0.2819 - acc: 0.9218 - val_loss: 0.1779 - val_acc: 0.9567
Epoch 2/10
60000/60000 [==============================] - 6s - loss: 0.1580 - acc: 0.9632 - val_loss: 0.1735 - val_acc: 0.9642
Epoch 3/10
60000/60000 [==============================] - 6s - loss: 0.1357 - acc: 0.9714 - val_loss: 0.1831 - val_acc: 0.9674
Epoch 4/10
60000/60000 [==============================] - 6s - loss: 0.1223 - acc: 0.9756 - val_loss: 0.1904 - val_acc: 0.9693
Epoch 5/10
60000/60000 [==============================] - 6s - loss: 0.1125 - acc: 0.9796 - val_loss: 0.2022 - val_acc: 0.9733
Epoch 6/10
60000/60000 [==============================] - 6s - loss: 0.1086 - acc: 0.9816 - val_loss: 0.2072 - val_acc: 0.9718
Epoch 7/10
60000/60000 [==============================] - 6s - loss: 0.1090 - acc: 0.9830 - val_loss: 0.2274 - val_acc: 0.9726
Epoch 8/10
60000/60000 [==============================] - 6s 

In [63]:
model = Sequential()
model.add(Dense(200, input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))
sgd = optimizers.Adagrad()
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [64]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=10,
          verbose=1, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 6s - loss: 0.2684 - acc: 0.9244 - val_loss: 0.1748 - val_acc: 0.9485
Epoch 2/10
60000/60000 [==============================] - 6s - loss: 0.1534 - acc: 0.9570 - val_loss: 0.1371 - val_acc: 0.9608
Epoch 3/10
60000/60000 [==============================] - 6s - loss: 0.1238 - acc: 0.9656 - val_loss: 0.1215 - val_acc: 0.9648
Epoch 4/10
60000/60000 [==============================] - 6s - loss: 0.1064 - acc: 0.9701 - val_loss: 0.1130 - val_acc: 0.9671
Epoch 5/10
60000/60000 [==============================] - 6s - loss: 0.0950 - acc: 0.9740 - val_loss: 0.1010 - val_acc: 0.9703
Epoch 6/10
60000/60000 [==============================] - 5s - loss: 0.0863 - acc: 0.9765 - val_loss: 0.0969 - val_acc: 0.9708
Epoch 7/10
60000/60000 [==============================] - 6s - loss: 0.0793 - acc: 0.9784 - val_loss: 0.0933 - val_acc: 0.9723
Epoch 8/10
60000/60000 [==============================] - 6s 

In [65]:
model = Sequential()
model.add(Dense(200, input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))
sgd = optimizers.Adadelta()
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [66]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=10,
          verbose=1, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 11s - loss: 0.4299 - acc: 0.8870 - val_loss: 0.2425 - val_acc: 0.9310
Epoch 2/10
60000/60000 [==============================] - 10s - loss: 0.2151 - acc: 0.9403 - val_loss: 0.1811 - val_acc: 0.9482
Epoch 3/10
60000/60000 [==============================] - 9s - loss: 0.1674 - acc: 0.9535 - val_loss: 0.1461 - val_acc: 0.9571
Epoch 4/10
60000/60000 [==============================] - 7s - loss: 0.1389 - acc: 0.9610 - val_loss: 0.1268 - val_acc: 0.9636
Epoch 5/10
60000/60000 [==============================] - 7s - loss: 0.1195 - acc: 0.9665 - val_loss: 0.1127 - val_acc: 0.9682
Epoch 6/10
60000/60000 [==============================] - 7s - loss: 0.1051 - acc: 0.9710 - val_loss: 0.1063 - val_acc: 0.9701
Epoch 7/10
60000/60000 [==============================] - 6s - loss: 0.0936 - acc: 0.9739 - val_loss: 0.0988 - val_acc: 0.9699
Epoch 8/10
60000/60000 [==============================] - 7

In [67]:
model = Sequential()
model.add(Dense(200, input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))
sgd = optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [68]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=10,
          verbose=1, validation_data=(X_test, Y_test))

/Users/u.nikulin/anaconda3/lib/python3.6/site-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s - loss: 0.2980 - acc: 0.9175 - val_loss: 0.1546 - val_acc: 0.9562
Epoch 2/10
60000/60000 [==============================] - 6s - loss: 0.1313 - acc: 0.9628 - val_loss: 0.1108 - val_acc: 0.9677
Epoch 3/10
60000/60000 [==============================] - 6s - loss: 0.0906 - acc: 0.9733 - val_loss: 0.0905 - val_acc: 0.9717
Epoch 4/10
60000/60000 [==============================] - 6s - loss: 0.0674 - acc: 0.9802 - val_loss: 0.0770 - val_acc: 0.9758
Epoch 5/10
60000/60000 [==============================] - 6s - loss: 0.0522 - acc: 0.9848 - val_loss: 0.0733 - val_acc: 0.9772
Epoch 6/10
60000/60000 [==============================] - 6s - loss: 0.0420 - acc: 0.9877 - val_loss: 0.0713 - val_acc: 0.9789
Epoch 7/10
60000/60000 [==============================] - 7s - loss: 0.0327 - acc: 0.9905 - val_loss: 0.0649 - val_acc: 0.9797
Epoch 8/10
60000/60000 [==============================] - 6s 

# Соберём всё вместе

In [71]:
model = Sequential()
model.add(Dense(200, kernel_initializer='he_uniform', input_shape=(784,)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dense(10))
model.add(Activation('softmax'))
sgd = optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=20,
          verbose=1, validation_data=(X_test, Y_test),
         callbacks=[callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)])

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 10s - loss: 0.2325 - acc: 0.9317 - val_loss: 0.1179 - val_acc: 0.9636
Epoch 2/20
60000/60000 [==============================] - 8s - loss: 0.0952 - acc: 0.9726 - val_loss: 0.0937 - val_acc: 0.9703
Epoch 3/20
60000/60000 [==============================] - 8s - loss: 0.0623 - acc: 0.9820 - val_loss: 0.0854 - val_acc: 0.9730
Epoch 4/20
60000/60000 [==============================] - 8s - loss: 0.0468 - acc: 0.9862 - val_loss: 0.0752 - val_acc: 0.9759
Epoch 5/20
60000/60000 [==============================] - 8s - loss: 0.0352 - acc: 0.9894 - val_loss: 0.0738 - val_acc: 0.9780
Epoch 6/20
60000/60000 [==============================] - 8s - loss: 0.0283 - acc: 0.9916 - val_loss: 0.0719 - val_acc: 0.9785
Epoch 7/20
60000/60000 [==============================] - 7s - loss: 0.0233 - acc: 0.9928 - val_loss: 0.0824 - val_acc: 0.9750
Epoch 8/20
60000/60000 [==============================] - 7s